In [2]:
#initialize
n = 4
N=binomial(n,2)

#list of unordered pairs of elements in {1,...,n}
def pairs(n): return flatten([[[i+1,j+1] for j in range(i+1,n)] for i in range(n-1)],max_level=1)

#sigma acts on an unordered pair
def act(sigma, p): return sorted([sigma(p[0]),sigma(p[1])])

#sigma acts as matrix in basis B on a vector v
def act_vect(sig,v,B=identity_matrix(N)): return (B.inverse()*sig.matrix()*B)*v

#permutation in Sigma_N which induced from perm in Sigma_n
def embed(sigma,n): return perm_from_sort([act(sigma,p) for p in pairs(n)])

#find permutation which orders a list L	
def perm_from_sort(L): return Permutation([pair[0] for pair in sorted(enumerate(L, 1), key=lambda x: x[1])]).to_cycles()

#image of generators of Sigma_n under embedding into Sigma_N
def embed_gens(n): return [embed(gen,n) for gen in SymmetricGroup(n).gens()]

In [3]:
#define the finite group
G = PermutationGroup(embed_gens(n)) #symmetric group \sigma_n as a subgroup of \sigma_N, with 2 generators

In [4]:
#optionally, the group could be any finite matrix group, but N must match
#G = MatrixGroup(matrix(QQ,[[1,0,0],[0,1,0],[0,0,1]]))

In [5]:
#fundamental domain given arbitrary distinct vector l and basis B
def fund_domain(l=[i for i in range(N)],B=identity_matrix(N),br=QQ):

    #augmented matrix for half-plane ineqs
    A = [ [l[j] - act_vect(G[i],vector(l),B)[j] for j in range(N)] for i in range(G.order())]
    b_1 = [0 for i in range(G.order())]
    bA = matrix(br,b_1).transpose().augment(matrix(br,A))

    #augmented matrix for positivity ineqs
    pos = B.inverse().transpose()
    b_2 = [0 for i in range(N)]
    bPos = matrix(br,b_2).transpose().augment(pos)

    #augmented matrix for fund. domain
    aug_l = bA.stack(bPos)
    
    #find convex polyhedral region, i.e. intersection of all ineqs
    poly = Polyhedron(ieqs=aug_l,base_ring=br)

    return([aug_l,poly])

In [6]:
#slice the fundamental domain with the plane x_1 + ... + x_N = 1
def cross_section(region,slice_dir=[1 for i in range(N)],chi=1,br=None):
    if br==None: br=region.base_ring()
    return Polyhedron(ieqs=region.inequalities(),eqns=[[-1]+slice_dir],base_ring=br)

In [63]:
#orbit of point x under group action
def orb(x): return list(set([tuple(act_vect(g,vector(x))) for g in G]))

#stabilizer of point x under group action
def stab(x): return list([g for g in G if act_vect(g,vector(x))==vector(x)])

#return orbit rep of x lying in F
def fund_domain_rep(x, F):
    for gx in orb(x):
        if F.contains(gx): return gx.list()

#size of stabilizer of point x
def stab_size(x): return G.order()/len(orb(x))

#returns interior point of polyhedron = average of vertices and rays 
#whose convex hull is polyhedron
def int_point(poly): 
    vert_rays = poly.vertices_list()+poly.rays_list()
    p = (1/len(vert_rays))*sum(vector(vr) for vr in vert_rays)
    #check
    if poly.relative_interior_contains(p): return p
    else: return "Failed."

#check if two faces of a polyhedron are glued together by group action
#by checking if the centroid is mapped to the interior of the other face
def faces_glued(f1,f2):
    p1 = int_point(f1.as_polyhedron())
    poly2 = f2.as_polyhedron()
    #check the orbit G.p1 and return immediately
    for g in G:
        if poly2.relative_interior_contains(act_vect(g,p1)): return True
    return False

In [8]:
#group element which glues face f1 to face f2
def gluing_elmts(f1,f2):
    p1 = int_point(f1.as_polyhedron())
    poly2 = f2.as_polyhedron()
    elmts = []
    for g in G:
        if poly2.relative_interior_contains(act_vect(g,vector(p1))): elmts.append(g)
    return elmts

In [9]:
#list of all k-dim'l faces of polyhedron F up to max_dim
#max_dim is dimension of F if not specified
def glued_face_lattice(F,max_dim=None):
    #set max_dim to be the dimension of F if not specified
    if max_dim is None:
        max_dim=F.dimension()
        
    glued_face_lat = []
    for k in range(max_dim+1):
        unique_k_faces = []
        for face in F.faces(k):
            #append face to unique_k_faces if it is not glued to any
            if not any(faces_glued(face,unique_face) for unique_face in unique_k_faces):
                unique_k_faces.append(face)
        #append unique k-faces to list of unique faces
        glued_face_lat.append(unique_k_faces) 
    return glued_face_lat

In [10]:
#returns the Euler characteristic of a space
def euler_char(F): return sum([(-1)^k*len(F.faces(k)) for k in range(F.dim()+1)])

#returns orbifold Euler characteristic given fundamental domain F for orbifold X/G.
#takes into account glued faces as to not include extra terms in alternating sum.
def orb_euler_char(glued_lat):
    #initialize alternating sum
    alt_sum = 0
    #loop over faces of each dim. k
    for k in range(len(glued_lat)):
        for f in glued_lat[k]:
            #convert face to polyhedron object
            f_poly = f.as_polyhedron()
            #choose arbitrary interior point of face
            p = int_point(f_poly)
            #add summand
            alt_sum += (-1)^k*(1/stab_size(p))
    return alt_sum

In [11]:
#construct vector space containing polyhedron face
def tangent_vs(face):
    face_rays = [vector(v) for v in face.rays()]
    face_verts = [vector(v) for v in face.vertices()]
    diff = [[v2-v1 for v2 in face_verts] for v1 in face_verts]
    flat_diff = [item for sublist in diff for item in sublist]
    return span(face_rays+flat_diff)

In [12]:
#induced orientation of a boundary face
def outward_normal(bndry_face,face):
    #construct vector space containing face
    V_face=tangent_vs(face)
    #construct vector space containing bndry_face
    V_bndry=tangent_vs(bndry_face)
    #construct unit normal vector n as orthogonal complement of V_bndry in V_face
    n=V_bndry.basis_matrix().right_kernel().intersection(V_face).basis()[0]
    #choose outward pointing normal
    eps=10^(-32)
    p=int_point(bndry_face.as_polyhedron())
    s_plus= p+eps*n
    s_minus= p-eps*n
    if face.as_polyhedron().relative_interior_contains(s_plus): return -n
    if face.as_polyhedron().relative_interior_contains(s_minus): return n
    
    print('Could not find outward pointing normal vector.')

In [13]:
#returns the induced orientation of bndry_face in face using an
#outward pointing normal vector wrt face
def induced_orient(bndry_face,face):
    #construct v.s. containing 'face'
    V_face=tangent_vs(face)
    #construct pos. oriented basis for bndry_face wrt face
    V_bndry=tangent_vs(bndry_face)
    n=outward_normal(bndry_face,face)
    bndry_basis=V_bndry.basis()
    
    coord_vecs = [V_face.coordinate_vector(n)]+[V_face.coordinate_vector(b) for b in bndry_basis]
    if matrix(coord_vecs).det() > 0: 
        if len(bndry_basis)==0: orient_basis = 1
        else: orient_basis = bndry_basis
    if matrix(coord_vecs).det() < 0: 
        if len(bndry_basis)==0: orient_basis = -1
        else: 
            orient_basis = [-bndry_basis[0]] + [bndry_basis[i] for i in range(1,len(bndry_basis))]
    
    return orient_basis

In [14]:
#returns boundary of polyhedron face inside F
def bndry(face,glued_lat,F):
    k=face.dim()
    
    #compute boundary of face before gluing
    face_bndry=[]
    for t in face.as_polyhedron().faces(k-1):
        for s in F.faces(k-1):
            if t.as_polyhedron()==s.as_polyhedron():
                face_bndry.append(s)

    bndry_vec=[0]*len(glued_lat[k-1])
    for f in face_bndry:
        for g in glued_lat[k-1]:
            #compute all group elements which map f to g
            sigma=gluing_elmts(f,g)
            if len(sigma) > 0:
                #tangent space to representative g of [f]=[g]
                V_g = tangent_vs(g)
                #tangent space to face f
                V_f = tangent_vs(f)
                
                #check if gluing is orientation preserving or reversing
                #by mapping the arbitrary basis assigned to f to the arbitrary
                #basis assigned to g. need to do this for each \sigma that maps
                #f to g. there are #stab([f]) of these elements. we then divide out
                #by this number to 'average' in the 'orbifold' sense
                orient_pres = 0
                for sig in sigma:
                    glued_basis = [act_vect(sig,v) for v in V_f.basis()]
                    glued_coord = [V_g.coordinate_vector(b) for b in glued_basis]
                    orient_pres += sign(matrix(glued_coord).det())
                orient_pres = orient_pres/len(sigma)
                    
                
                #check whether the induced orientation of f matches the arbitrary
                #orientation assigned to f. dimension 0 is a special case.
                induced_f_orient = induced_orient(f,face)
                if f.dim() == 0: 
                    induced_match = induced_f_orient
                else: 
                    f_coord = [V_f.coordinate_vector(b) for b in induced_f_orient]
                    induced_match = sign(matrix(f_coord).det())
                
                #update the number of times the glued face g occurs in the boundary
                #accounting for signs
                indx = glued_lat[k-1].index(g)
                bndry_vec[indx] += orient_pres*induced_match
    
    return bndry_vec

In [15]:
#boundary map
def bndry_map(k,glued_lat,F): return matrix(ZZ,[bndry(f,glued_lat,F) for f in glued_lat[k]]).transpose()

#cellular homology complex
def cell_chain_cmplx(glued_lat,F): return ChainComplex({k:bndry_map(k,glued_lat,F) for k in range(1,len(glued_lat))},degree=-1)

#boundary map in Z/2Z
def mod2_bndry_map(k,glued_lat,F): return matrix(Integers(2),[bndry(f,glued_lat,F) for f in glued_lat[k]]).transpose()

#cellular homology in Z/2Z
def mod2_cell_chain_cmplx(glued_lat,F): return ChainComplex({k:mod2_bndry_map(k,glued_lat,F) for k in range(1,len(glued_lat))},degree=-1)

In [16]:
#compute a fundamental domain
F=fund_domain()[1]; F
#F is the convex hull of the origin and some rays
F.rays()

(A ray in the direction (0, 1, 0, 0, 1, 0),
 A ray in the direction (0, 0, 1, 1, 0, 0),
 A ray in the direction (1, 0, 0, 0, 0, 1),
 A ray in the direction (0, 0, 0, 0, 0, 1),
 A ray in the direction (0, 0, 0, 1, 1, 1),
 A ray in the direction (0, 0, 0, 0, 1, 1),
 A ray in the direction (0, 0, 1, 0, 1, 1))

In [17]:
#optionally specify a centering vector, l=[x1,...,xN]
l=[floor(sqrt(i)) for i in range(N)]
F_l=fund_domain(l)[1]; F_l
#the fundamental changes with l. locally it may be trivial, but it is complicated with wall crossings
F_l.rays()

(A ray in the direction (0, 0, 0, 0, 0, 1),
 A ray in the direction (0, 0, 1, 1, 1, 1),
 A ray in the direction (0, 0, 0, 0, 1, 0),
 A ray in the direction (0, 0, 1, 0, 1, 1),
 A ray in the direction (0, 1, 1, 1, 1, 0),
 A ray in the direction (0, 0, 0, 1, 1, 1),
 A ray in the direction (0, 1, 0, 1, 1, 1),
 A ray in the direction (0, 1, 0, 0, 1, 0),
 A ray in the direction (0, 1, 1, 0, 1, 1),
 A ray in the direction (1, 1, 1, 1, 1, 1),
 A ray in the direction (1, 1, 0, 0, 1, 1))

In [18]:
#slice a cross section w/ sum of edge weights = 1. optionally specify constant.
F2=cross_section(F); F2.vertices()

(A vertex at (1/2, 0, 0, 0, 0, 1/2),
 A vertex at (0, 1/2, 0, 0, 1/2, 0),
 A vertex at (0, 0, 0, 0, 1/2, 1/2),
 A vertex at (0, 0, 0, 1/3, 1/3, 1/3),
 A vertex at (0, 0, 1/2, 1/2, 0, 0),
 A vertex at (0, 0, 1/3, 0, 1/3, 1/3),
 A vertex at (0, 0, 0, 0, 0, 1))

In [19]:
#compute the number of faces for the sliced fundamental domain
[len(F2.faces(k)) for k in range(6)]

[7, 19, 26, 19, 7, 1]

In [20]:
#compute the size of an interior point on a face in F2
set([stab_size(int_point(f.as_polyhedron())) for f in F2.faces(1)])

{1, 2, 4, 8}

In [21]:
#in 2 or 3d, plot the sliced fundamental domain
#F2.show()

In [22]:
#use a finer triangulation
F3=F2.barycentric_subdivision(subdivision_frac=1/6); F3

A 5-dimensional polyhedron in QQ^6 defined as the convex hull of 78 vertices (use the .plot() method to plot)

In [23]:
F3.faces(2)

(A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyhedron in QQ^6 defined as the convex hull of 3 vertices,
 A 2-dimensional face of a Polyh

In [24]:
#show the number of faces of each dimension
[len(F3.faces(k)) for k in range(N)]

[78, 708, 2072, 2400, 960, 1]

In [25]:
#compute the size of an interior point on a face in F3
set([stab_size(int_point(f.as_polyhedron())) for f in F3.faces(1)])

{1}

In [26]:
#print a list of 1-faces which are glued together
for f1 in F2.faces(1):
    for f2 in F2.faces(1):
        if faces_glued(f1,f2):
            print(F2.faces(1).index(f1),F2.faces(1).index(f2))

0 0
1 1
2 2
2 14
3 3
4 4
5 5
5 10
5 16
6 6
7 7
8 8
8 11
8 17
9 9
9 13
10 5
10 10
10 16
11 8
11 11
11 17
12 12
12 15
12 18
13 9
13 13
14 2
14 14
15 12
15 15
15 18
16 5
16 10
16 16
17 8
17 11
17 17
18 12
18 15
18 18


In [27]:
#see if vertices of barycentric subdivision contain previous vertices
set(F3.vertices()).intersection(set(F2.vertices()))

set()

In [32]:
#compute group elements which glue faces together
gluing_elmts(F2.faces(1)[2],F2.faces(1)[14])

[(1,2)(5,6)]

In [ ]:
#print vertices of all 1-faces
for v in F2.faces(1):
    print(v.vertices())

In [ ]:
#compute the homology of the space F2 with gluings
glued_lat=glued_face_lattice(F2)
cell_chain_cmplx(glued_lat,F2).homology()

In [ ]:
#compute the orbifold Euler characteristic
orb_euler_char(glued_lat)

In [ ]:
#there is a bug in the boundary maps
#
#assigning random orientations seems okay, as long as you keep track of whether the induced orientation matches w/ sign.
#determining whether the gluing preserves orientation seems okay, giving another sign.
#taking into account the stabilizer size is necessary to get a complex.
#multiplying the signs is necessary to give a complex.
#
#however, the homology changes when you change the centering vector of the fund. domain. it should be invariant.
#
#only issue I can think of is if any faces are stabilized, then the celluar decomposition may be bad.
#can triangulate further to remove stabilizers, but then there are too many vertices to run the computation.
#what seems to be happening is that if you have a flip stabilizer, the face subspace is sometimes preserved,
#so when computing the boundary number of occurences of f in the boundary keeping track of orientation, you get 
#(f+f)/2=f, which is good. if it's a rotation, you get (f-f)/2 = 0, which effectively *removes* the boundary face f.
#
#solution: only want H_1, so just truncate the complex. even so, the matrix would be ~ 700x2000, which is too big.
#needs to be around 200x200 total entries to compute kernels effectively. it may be possible to speed up the computation 
#somewhere, or carefully subdivide without adding too many extra cells. barycentric subdivision pulls out vertices in a
#way that doesn't leave the original vertices intact - not sure what the deal is there.

In [ ]:
#compute the face lattice, taking into account gluing of faces
#glued_lat=glued_face_lattice(F3,max_dim=0)
#truncate to compute H_1 using d1, d2
#d1=bndry_map(1,glued_lat,F3)
#ChainComplex({1:d1},degree=-1).homology()

In [66]:
stab(int_point(F2.faces(2)[12].as_polyhedron()))

[(), (1,4)(3,6)]

In [61]:
F2.faces(2)[12].vertices()

(A vertex at (1/2, 0, 0, 0, 0, 1/2),
 A vertex at (0, 0, 1/2, 1/2, 0, 0),
 A vertex at (0, 0, 1/3, 0, 1/3, 1/3))